## Accuracy as a function taxon abundance

### Variable parameters:

* Partition communities into 'dominant' and 'rare' subsets
  * __dominant/rare cutoff: 1% relative abundance?__
* Assess sensitivity & specificity for each subset
* __Note:__ 
  * Much less time to perform if using data from past simulation run datasets

### Analysis:

* Sup Figures:
  * Same as above simulation runs, but split into 'dominant' vs 'rare'

# User variables

In [1]:
workDir = '/home/nick/notebook/SIPSim/dev/bac_genome1210/'
simDir = os.path.join(workDir, 'atomIncorp_taxaIncorp')
buildDir = os.path.join(workDir, 'atomIncorp_taxaIncorp_abund')
R_dir = '/home/nick/notebook/SIPSim/lib/R/'

# Init

In [2]:
import glob
from os.path import abspath
import nestly
from IPython.display import Image, display

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
library(ggplot2)
library(dplyr)
library(tidyr)
library(gridExtra)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: grid


In [5]:
if not os.path.isdir(buildDir):
    os.makedirs(buildDir)

# Analyzing accuracy info

# SANDBOX

In [7]:
%%R -i workDir
setwd(workDir)

deseq.res = readRDS('./validation/OTU_n2_abs1e10_sub20000_DESeq2')

In [17]:
%%R

comm = read.delim('./validation/comm.txt', sep='\t')
comm.s = comm %>%
    group_by(taxon_name) %>%
    summarize(mean_rel_abund_perc = mean(rel_abund_perc))
comm.s %>% head

Source: local data frame [6 x 2]

                            taxon_name mean_rel_abund_perc
1       Acaryochloris_marina_MBIC11017         0.089773401
2       Acetobacterium_woodii_DSM_1030         0.013390418
3 Acetobacter_pasteurianus_IFO_3283-12         0.003064792
4    Acetohalobium_arabaticum_DSM_5501         0.058114601
5         Acholeplasma_laidlawii_PG-8A         0.004121299
6        Achromobacter_xylosoxidans_A8         0.024560238


In [21]:
%%R
tmp = deseq.res %>%
    as.data.frame() 
tmp$taxon_name = rownames(tmp)

tmp = inner_join(tmp, comm.s, c('taxon_name' = 'taxon_name'))
tmp = tmp %>%
    mutate(dom_rare = mean_rel_abund_perc > 1)

tmp %>% head

    baseMean log2FoldChange    lfcSE      stat    pvalue padj         p padj.BH
1  0.4745836      0.9200834 1.394946 0.4803651 0.3154839    1 0.3154839       1
2  3.2902741     -4.6553395 1.275818 0.0000000 0.9999397    1 0.9999397       1
3  4.8031896     -1.0010777 1.176846 0.0000000 0.8561264    1 0.8561264       1
4  0.7088409      0.5138864 1.270017 0.2077818 0.4176997    1 0.4176997       1
5 28.0436962     -1.2126332 1.274566 0.0000000 0.8744236    1 0.8744236       1
6 29.5094790      0.5352219 1.697529 0.1680218 0.4332831    1 0.4332831       1
                         taxon_name mean_rel_abund_perc dom_rare
1 Acetohalobium_arabaticum_DSM_5501          0.05811460    FALSE
2       Acidovorax_citrulli_AAC00-1          0.07152763    FALSE
3            Acidovorax_ebreus_TPSY          0.14267760    FALSE
4      Aeromonas_hydrophila_AL09-71          0.02364744    FALSE
5  Alkalilimnicola_ehrlichii_MLHE-1          0.77596664    FALSE
6    Alkaliphilus_oremlandii_OhILAs          1.292

In [22]:
%%R
tmp %>% filter(dom_rare == FALSE) %>% nrow %>% print
tmp %>% filter(dom_rare == TRUE) %>% nrow %>% print

[1] 215
[1] 17
